In [6]:
import os
import sys
import pandas as pd
import polars as pl

sys.path.append('../../data')
import kaggle_evaluation.cmi_inference_server
from preprocessing import CMIPreprocessor


In [7]:
# 前処理器を初期化
preprocessor = CMIPreprocessor()
preprocessor.load_preprocessors()
preprocessor.load_model()

print("✅ 前処理器の初期化が完了しました")


GPU利用可能: 1台
✅ センサーScaler読み込み完了
✅ DemographicsScaler読み込み完了
✅ PCA Transformer作成完了
  元の次元数: 18
  変換後次元数: 18
  累積寄与率: 1.0000
モデル読み込み中: /mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/final_model_20250709_085324.keras
✅ モデル読み込み完了
✅ 前処理器の初期化が完了しました


/mnt/c/Users/ShunK/works/CMI_comp/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 26 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    テスト用の予測関数
    
    Parameters:
    -----------
    sequence : pl.DataFrame
        センサーデータのシーケンス
    demographics : pl.DataFrame
        Demographics情報
    
    Returns:
    --------
    str : 予測されたジェスチャー名
    """
    # Polars DataFrameをPandas DataFrameに変換
    sequence_df = sequence.to_pandas()
    demographics_df = demographics.to_pandas()
    
    # 前処理器を使用して予測
    return preprocessor.predict_gesture(sequence_df, demographics_df)


In [9]:
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # ローカル環境用のパス設定
    from pathlib import Path
    
    current_dir = Path(__file__).parent if '__file__' in globals() else Path('.')
    data_dir = current_dir.parent.parent / 'data'
    
    inference_server.run_local_gateway(
        data_paths=(
            '../../data/test.csv',
            '../../data/test_demographics.csv',
        )
    )


推論開始: sequence_id=SEQ_000011
  センサーデータ形状: (51, 332)
  センサーカラム数: 332
⚠️  シーケンスが短いため、パディングを追加します。
  作成されたウィンドウ数: 1
  ウィンドウ形状: (1, 64, 332)
  正規化後センサーデータ形状: (1, 64, 332)
  Demographics特徴量形状: (20,)
  18特徴量に削減: (18,)
  PCA変換後形状: (1, 18)
  正規化後形状: (1, 18)
  Demographics特徴量形状: (1, 18)
  推論前センサーデータ統計:
    - 最小値: -3.449285
    - 最大値: 4.365560
    - 平均値: -0.420792
    - 標準偏差: 0.748877
  推論前Demographics統計:
    - 最小値: -4018.913574
    - 最大値: 275120.812500
    - 平均値: 16141.713867
    - 標準偏差: 63000.281250
⚠️  予測結果にNaN/Infが含まれています
  NaN数: 18
  Inf数: 0
  予測結果の形状: (1, 18)
  予測結果の最初の値: [nan nan nan nan nan]
  NaN対策: デフォルト予測を使用します
  修正後予測結果: [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556]
  単一ウィンドウ予測信頼度: 0.0556
  予測結果: Above ear - pull hair
推論開始: sequence_id=SEQ_000001
  センサーデータ形状: (56, 332)
  センサーカラム数: 332
⚠️  シーケンスが短いため、パディングを追加します。
  作成されたウィンドウ数: 1
  ウィンドウ形状: (1, 64, 332)
  正規化後センサーデータ形状: (1, 64, 332)
  Demographics特徴量形状: (20,)
  18特徴量に削減: (18,)
  PCA変換後形状: (1, 18)
  正規化後形状: (1, 18)
  Demogra

In [10]:
import pandas as pd
df = pd.read_parquet('submission.parquet')
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('First 10 rows:')
print(df.head(10))
print('Value counts:')
print(df['gesture'].value_counts())

Shape: (2, 2)
Columns: ['sequence_id', 'gesture']
First 10 rows:
  sequence_id                gesture
0  SEQ_000011  Above ear - pull hair
1  SEQ_000001  Above ear - pull hair
Value counts:
gesture
Above ear - pull hair    2
Name: count, dtype: int64
